이 노트에서는 도로 네트워크를 그래프로 가져오는 과정에서 발생한 문제를 살펴보고, 이를 해결하는 과정을 다룹니다.
순서는 다음과 같습니다.

도로 네트워크를 그래프로 가져오면 어떤 문제가 발생하는걸까?
데이터 하나씩 확인하며 문제 해결하기

In [1]:
# 출력되는 모든 값 표시
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

도로 네트워크를 그래프로 가져오면 어떤 문제가 발생하는걸까?
먼저 도로 네트워크 데이터를 가져옵니다.

In [7]:
import geopandas as gpd

df_link = gpd.read_file('data/moc_link_2017.geojson')
df_link.shape
df_link.head()

DriverError: data/moc_link_2017.geojson: No such file or directory

In [ ]:
중간에 None 값이 있습니다.
None 값의 노드를 임의의 노드이름으로 수정해줍니다.

In [ ]:
df_link[(df_link['f_node'].isnull()) | (df_link['t_node'].isnull())]

In [ ]:
df_link.loc[5636, 't_node'] = 'temp1'
df_link.loc[5638, 'f_node'] = 'temp1'

In [ ]:
import networkx as nx

# edge 들로 그래프를 구성합니다.
graph_proj = nx.from_pandas_edgelist(df_link, 
                                     source='f_node', target='t_node',
                                     create_using=nx.MultiDiGraph)


# node 들에 (x, y) 정보를 넣습니다.
d = {}
for row in df_link.itertuples():
    f_node = getattr(row, 'f_node')
    t_node = getattr(row, 't_node')
    geometry = getattr(row, 'geometry')
    
    m_line_string = getattr(row, 'geometry')
    f_node_coord = list(m_line_string[0].coords[0])
    t_node_coord = list(m_line_string[-1].coords[-1])
    
    d[f_node] = dict(x=f_node_coord[0], y=f_node_coord[1])
    d[t_node] = dict(x=t_node_coord[0], y=t_node_coord[1])
    
nx.set_node_attributes(graph_proj, d)

In [ ]:
import os
import deckgljupyter.Layer as deckgl

access_token = os.getenv('MAPBOX_ACCESS_TOKEN')
view_options = {
    'center': [126.908, 37.132],
    'zoom': 12,
    'bearing': 0,
    'pitch': 60,
    'style': 'mapbox://styles/mapbox/dark-v9',
    'access_token': access_token
}

def draw_graph(graph):
    # make node list
    node_list = []
    for node in graph.nodes:
        node_list.append({
            'node': node,
            'position': list(graph.nodes[node].values())
        })

    # make edge list
    link_list = []
    for edge in graph.edges:
        # edge is like ('2330045500', '2330045200', 0).
        u, v = edge[0], edge[1]
        link_list.append({
            'edge': "{} -> {}".format(u, v),
            'path': [list(graph.nodes[u].values()), list(graph.nodes[v].values())]
        })
        
    # draw map
    m = deckgl.Map(**view_options)

    m.add(deckgl.ScatterplotLayer(node_list,
                               getRadius=10,
                               getFillColor=[225, 120, 0],
                               pickable=True,
                               tooltip=['node']))

    m.add(deckgl.PathLayer(link_list,
                           getWidth=3,
                           getColor=[255, 221, 0],
                           opacity=0.5,
                           pickable=True,
                           tooltip=['edge']))

    m.show()

In [ ]:
draw_graph(graph_proj)

In [ ]:
df_link[df_link.f_node == '2230027900']

In [ ]:
마지막 geometry 열을 살펴보면, 각 링크에서 2230027900 노드의 좌표를 알 수 있습니다.

In [ ]:
for i in df_link[df_link.f_node == '2230027900']['geometry']:
    print(list(i[0].coords)[0])

In [ ]:
원래대로라면, 위 6개의 좌표가 모두가 같아야하는데, 그렇지 않습니다.
앞의 3개와 뒤에 3개의 좌표가 다릅니다.
즉, 위의 문제는 다음과 같이 유추할 수 있습니다.

원래 데이터 자체에 오류가 있어서,
이를 그래프로 옮기는 과정에서 잘못 처리되었다.
즉, 원래는 다른 하나의 좌표가 나머지 좌표를 덮어버려서 위와 같이 노드하나가 소실된 것으로 보입니다.

그럼 2개 중(앞의 3개 vs 뒤 3개) 어떤 좌표를 선택해야 할까요?
두 번째 스크린샷을 보면 유추할 수 있습니다.
2230027900 노드의 자리는 원래 첫 번째 스크린 샷의 자리가 아닙니다.
고속도로 쪽 노드가 2230027800 임을 감안하면, 그 다음에 2230027900 가 등장하는 것이 맞는 것을 추론할 수 있습니다.

첫 번째 스크린 샷에 등장하는 또 다른 노드 2330120100 역시 마찬가지입니다.

In [ ]:
df_link[df_link.t_node == '2330120100']
for i in df_link[df_link.f_node == '2330120100']['geometry']:
    print(list(i[0].coords)[0])

In [ ]:
정리하면, df_link 즉 원래 데이터 자체에 노드 정보가 잘못 기입된 것입니다.
2개의 노드번호만 오류가 있으므로, 직접 이 노드들을 수정하여 처리해보겠습니다.

데이터 하나씩 확인하며 수정하기
스크린 샷 1번에 나온 두 노드의 번호는 각각 2230027900 와 2330120100 가 아닙니다.
(아까도 말했지만, 데이터 상에는 그렇게 기입되있지만, 시각화 해보면 잘못 기입된 것을 추론할 수 있습니다.)

따라서, 이 두 노드의 번호를 임의 다른 번호로 명명하여, 원래 노드가 분실되지 않게 처리하겠습니다.

2230027900 노드

In [ ]:
df_link[df_link.f_node == '2230027900']

In [ ]:
마지막 3개의 링크에서 f_node 가 잘못되었습니다.
실제 노드의 이름을 모르므로, temp2 라고 수정하겠습니다.

In [ ]:
df_link.loc[[7075, 7079, 7086], 'f_node'] = 'temp2'

In [ ]:
df_link[df_link.t_node == '2230027900']

In [ ]:
마찬가지로 마지막 3개에서 t_node 를 수정합니다.

In [ ]:
2330120100 노드
위와 동일한 작업을 합니다.



In [ ]:
df_link[df_link.f_node == '2330120100']
df_link.loc[[7076, 7084, 7085], 'f_node'] = 'temp3'
df_link[df_link.t_node == '2330120100']
df_link.loc[[7078, 7081, 7086], 't_node'] = 'temp3'

In [ ]:
해결했는지 확인하기
이제 다시 그래프를 구성하여, 시각화함으로써 제대로 해결이 되었는지 확인해보겠습니다.

In [ ]:
# edge 들로 그래프를 구성합니다.
graph_proj = nx.from_pandas_edgelist(df_link, 
                                     source='f_node', target='t_node',
                                     create_using=nx.MultiDiGraph)


# node 들에 (x, y) 정보를 넣습니다.
d = {}
for row in df_link.itertuples():
    f_node = getattr(row, 'f_node')
    t_node = getattr(row, 't_node')
    geometry = getattr(row, 'geometry')
    
    m_line_string = getattr(row, 'geometry')
    f_node_coord = list(m_line_string[0].coords[0])
    t_node_coord = list(m_line_string[-1].coords[-1])
    
    d[f_node] = dict(x=f_node_coord[0], y=f_node_coord[1])
    d[t_node] = dict(x=t_node_coord[0], y=t_node_coord[1])
    
nx.set_node_attributes(graph_proj, d)
draw_graph(graph_proj)
#아까 오류가 있던 부분만 캡처해보면, 다음과 같이 잘 해결한 것을 볼 수 있습니다.


In [ ]:
정리
정리하면, 기존 데이터 df_link 의 총 3개의 노드에서 노드번호가 잘못 기입된 오류가 있었습니다.
이를 보정하는 코드만 정리는 다음과 같았습니다.

df_link.loc[5636, 't_node'] = 'temp1'
df_link.loc[5638, 'f_node'] = 'temp1'
df_link.loc[[7075, 7079, 7086], 'f_node'] = 'temp2'
df_link.loc[[7082, 7085, 7087], 't_node'] = 'temp2'
df_link.loc[[7076, 7084, 7085], 'f_node'] = 'temp3'
df_link.loc[[7078, 7081, 7086], 't_node'] = 'temp3'